In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from fractions import Fraction
from natsort import natsorted
from ftfy import fix_text
import time
today = time.strftime("%Y_%m_%d")
import collections
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit  
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

def three(usa,reg):
    trip=fr'''(?i){reg}|()''' 
    usa['m_name']=usa['product_name'].apply(lambda x: re_extract(trip,str(x)))
    usa['m_desc']=usa['long_desc'].apply(lambda x: re_extract(trip,str(x)))
    usa['m_custom']=usa['custom_fields'].apply(lambda x: re_extract(trip,str(x)))
    imported=usa[(usa['m_name'].astype(str)!='[]')|(usa['m_desc'].astype(str)!='[]')|(usa['m_custom'].astype(str)!='[]')]
    impor=usa[(usa['m_name'].astype(str)=='[]')&(usa['m_desc'].astype(str)=='[]')&(usa['m_custom'].astype(str)=='[]')]
    print('values: '+str(len(imported)))
    print('no values: '+str(len(impor)))
    return imported,impor

def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

# Brackets

In [2]:
customer_id = '5'
customer_name='%bedbathandbeyond%'
dateszs='2023-04-20'

params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs}
bbby = query_from_file(file_name='./query/curated_all_attributes_all_customers_brackets.sql', params=params)
print(len(bbby))

import time
today = time.strftime("%Y_%m_%d")
print(today)

wbm=bbby
trip=r'''(?i)(\{)|(\})|()'''                                                              
wbm['match']=wbm['value'].apply(lambda x: re_extract(trip,str(x)))
bracket=wbm[(wbm['match'].astype(str)!='[]')&(wbm['snake_case_name'].astype(str)!='None')]#['value'].explode().value_counts()[0:500]
bracket['matches']=bracket['value'].apply(lambda x: re.sub(r'\{','[',str(x))).apply(lambda x: re.sub(r'\}',']',str(x)))
bracket['external_id'].explode().value_counts()

379379
2023_04_25


Series([], Name: external_id, dtype: int64)

In [3]:
bracket

Empty DataFrame
Columns: [value, customer_name, snake_case_name, external_id, Curated Date, Updated Date, match, matches]
Index: []

In [4]:
height=bracket[bracket['snake_case_name'].astype(str)=='height']
height['Q:height']=height['matches']
match_height=height[['external_id','Q:height']]

match_height

Empty DataFrame
Columns: [external_id, Q:height]
Index: []

In [5]:
width=bracket[bracket['snake_case_name'].astype(str)=='width']
width['Q:width']=width['matches']
match_width=width[['external_id','Q:width']]

match_width

Empty DataFrame
Columns: [external_id, Q:width]
Index: []

In [6]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY -_{get_df_name(matches)}-{today}match_height.csv',index=False) 
# looks_good('Bed Bath & Beyond', match_height,today)

In [7]:
bracket['external_id'].to_list()

[]

# Drop Length

In [8]:
customer_id = '5'
customer_name='%bedbathandbeyond%'
dateszs='2002-08-11'
attribut='drop_length'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribute':attribut}

dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
# dfz=dfs[dfs['custom_fields'].astype(str)!=None]
# custom_field_df=pd.json_normalize(dfs['custom_fields'])
# fields = ['LONG_DESCRIPTION']
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# df.drop_duplicates(subset='external_id', inplace=True)

In [9]:
df=dfs

In [10]:
pat=r'''(?i)(\d+\s?\\?(?:"|.inc?h?e?s?|'').?drop)|(drop in either.\d\d''.or.\d+'')|((?<=drop in either.)\d+(?:"|.inc?h?e?s?|''))|()'''
df['name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
df['desc']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
df['custom']=df['custom_fields'].apply(lambda x: re_extract(pat,str(x)))

na=df[(df['name'].astype(str)=='[]')&(df['desc'].astype(str)=='[]')&(df['custom'].astype(str)=='[]')]
drop=df[(df['name'].astype(str)!='[]')|(df['desc'].astype(str)!='[]')|(df['custom'].astype(str)!='[]')]

na[curation_col]='n/a'
print(len(df))
print(len(na))
print(len(drop))
matchna_table=na[['external_id',curation_col]]

265
257
8


In [11]:
drops=drop[drop['desc'].astype(str)!='[]']
print(len(drops))
drops[curation_col]=drops['desc'].apply(lambda x: re.sub(r'''(?i)(\s?(?:\\?(?<!\[)(?<!\,)"(?!\])(?!\,)|(?<!\[)(?<!\,)''(?!\])(?!\,)|inc?h?e?s?)\s?)''',' in',str(x))).apply(lambda x: re.sub(r"((?<=\[)')|('(?=\]))|('(?=,))|((?=,)\s')",'"',str(x))).apply(lambda x: re.sub(r'''(\s?')''','"',str(x))).apply(lambda x: re.sub(r'''(?i)(drop in either\s?)''','',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?ineither\s?)''','',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?drop\s?)|(\\)|(\-)''','',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?or\s?)''','","',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: '["'+'","'.join(natsorted(x))+'"]').apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))

8


In [12]:
rounding_inch_feet(drops, curation_col,'a-eghj-mo-su-z')
print(len(drops))
drops=drops[drops['rounding'].astype(str)=='[]']
print(len(drops))

8
0


In [13]:
match_drop=drops[['external_id',curation_col]]
match_drop_na=na[['external_id',curation_col]]
# drops[curation_col].explode().value_counts()

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_drop_na.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribut,match_drop_na,today)

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_drop.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribut,match_drop,today)

# Watt

In [15]:
customer_id = '5'
customer_name='%bedbathandbeyond%'
dateszs='2002-08-11'
attribut='wattage'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribute':attribut}

dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
dfz=dfs[dfs['custom_fields'].astype(str)!=None]
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)

df=dfs

del df['product_id']
del df['bucket_id']
del df['url']
del df['image_url']
del df['value']

In [16]:
pat=r'(?i)(\d+\s?W(?!\w)\b)|((?<!")(?<!"\s)\d+[^0-9"]{0,3}(?:max(?:imum)?)?[^0-9"]{0,3}watt(?:age)?(?!a)(?!\d)(?!\s\d))|(watt(?:age)[^0-9]{0,3}\d+)|()'
df['name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
df['desc']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
df['custom']=df['custom_fields'].apply(lambda x: re_extract(pat,str(x)))
na=df[(df['name'].astype(str)=='[]')&(df['desc'].astype(str)=='[]')&(df['custom'].astype(str)=='[]')]
watt=df[(df['name'].astype(str)!='[]')|(df['desc'].astype(str)!='[]')|(df['custom'].astype(str)!='[]')]
na[curation_col]='n/a'
print(len(df))
print(len(na))
print(len(watt))
matchna_wattage=na[['external_id',curation_col]]

831
145
686


In [17]:
# x[x['name'].astype(str)=='5\" Wattage']

In [18]:
watt[['product_name','long_desc','name','desc','custom']]
x=watt[watt['name'].astype(str)!='[]']
y=watt[(watt['name'].astype(str)=='[]')&(watt['desc'].astype(str)!='[]')]
z=watt[(watt['name'].astype(str)=='[]')&(watt['custom'].astype(str)!='[]')]

x[curation_col]=x['name'].apply(lambda x: re.sub(r'(?i)(Watta?g?e?)|([\s\-])','',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x))).apply(lambda x: re.sub(r'\,\s',',',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: '["'+'","'.join(natsorted(x))+'"]').apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))
del x['custom_fields']
x[curation_col]=x[curation_col].apply(lambda x: re.sub(r'(?i)((?<=\d)(?="))','W',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: '["'+'","'.join(natsorted(x))+'"]').apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))
# x[curation_col]#.explode().value_counts()

In [19]:
# print(len(x))
rounding_w(x, curation_col,'a-vx-z')
drops=x[x['rounding'].astype(str)=='[]']
print(len(x))
match_watt=x[['external_id',curation_col]]

11


In [20]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_watt.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribut,match_watt,today)

In [21]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matchna_wattage.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribut,matchna_wattage,today)

# Size Diameter Rug Runners

In [22]:
customer_id = '5'
customer_name='%bedbathandbeyond%'
dateszs='2002-08-11'
attribut='size_diameter_rugs_runners'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribute':attribut}

dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
dfz=dfs[dfs['custom_fields'].astype(str)!=None]
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)

In [23]:
pat=r'(?i)(round)|(\bring\b)'
df['name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
df['desc']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
df['custom']=df['LONG_DESCRIPTION'].apply(lambda x: re_extract(pat,str(x)))
na=df[(df['name'].astype(str)=='[]')&(df['desc'].astype(str)=='[]')&(df['custom'].astype(str)=='[]')]
na[curation_col]='n/a'
print(len(df))
print(len(na))
matchna_size_diameter_rugs_runners=na[['external_id',curation_col]]

729
700


In [24]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribut,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribut}-{get_df_name(matches)}-{today}-matchna_size_diameter_rugs_runners.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribut,matchna_size_diameter_rugs_runners,today)

[]


# Shelf Weight Capacity

In [25]:
customer_id = '5'
formatted_attribute = 'shelf_weight_capacity'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [26]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)
# reg_ex(df)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df=df[(df['external_id'].astype(str)!='200045713')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')&(df['external_id'].astype(str)!='200046141')&(df['external_id'].astype(str)!='200046271')]
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

232
[]


In [27]:
pat=r'(?i)(weight.?capacity(?:.{0,5}shelf)?)|(each.?shelf)|((?:upper|lower|top|middle|bottom).?(?:board|shelf))|(load.?capacity)|(\d+.?lb.{0,10}shel(?:ve|f))|(weight.?limit)|(shel(?:ve|f).{0,10}\d+.?lb)'
df['name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
df['desc']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
df['custom']=df['LONG_DESCRIPTION'].apply(lambda x: re_extract(pat,str(x)))
na=df[(df['name'].astype(str)=='[]')&(df['desc'].astype(str)=='[]')&(df['custom'].astype(str)=='[]')]
na[curation_col]='n/a'
print(len(na))
matchna_self_weight=na[['external_id',curation_col]]

58


In [28]:
# shelf=df[(df['name'].astype(str)!='[]')|(df['desc'].astype(str)!='[]')|(df['long_desc'].astype(str)!='[]')]
# shelf

In [29]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matchna_self_weight.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribute,matchna_self_weight,today)

[]


# Compartment Width

In [30]:
customer_id = '5'
formatted_attribute = 'compartment width'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [31]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)
# reg_ex(df)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df=df[(df['external_id'].astype(str)!='200045713')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')&(df['external_id'].astype(str)!='200046141')&(df['external_id'].astype(str)!='200046271')]
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

15
[]


In [32]:
pat=r'(?i)Cubbie.?Width.{0,3}\d+|compartment.?Width.{0,3}\d+'
df['name']=df['product_name'].apply(lambda x: re_extract(pat, str(x)))
df['desc']=df['long_desc'].apply(lambda x: re_extract(pat, str(x)))
dfna=df[(df['name'].astype(str)=='[]')&(df['desc'].astype(str)=='[]')]
dfna[curation_col]='n/a'
matchna_compartment_width=dfna[['external_id',curation_col]]

In [33]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribute,matchna_compartment_width,today)

[]


# Compartment Height

In [34]:
customer_id = '5'
formatted_attribute = 'compartment height'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [35]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)
# reg_ex(df)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df=df[(df['external_id'].astype(str)!='200045713')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')&(df['external_id'].astype(str)!='200046141')&(df['external_id'].astype(str)!='200046271')]
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

15
[]


In [36]:
pat=r'(?i)Cubbie.?height.{0,3}\d+|compartment.?height.{0,3}\d+'
df['name']=df['product_name'].apply(lambda x: re_extract(pat, str(x)))
df['desc']=df['long_desc'].apply(lambda x: re_extract(pat, str(x)))
dfna=df[(df['name'].astype(str)=='[]')&(df['desc'].astype(str)=='[]')]
dfna[curation_col]='n/a'
matchna_compartment_height=dfna[['external_id',curation_col]]

In [37]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribute, matchna_compartment_height,today)

[]


# Capacity Oz

In [38]:
customer_id = '5'
formatted_attribute = 'capacity_oz'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])

In [39]:
pat=r'(?i)((?:capacity)[^\w]{0,3}\d+(?:\.\d+)?[\s\-]?(?:oz|ounces))|(\d+(?:\.\d+)?[\s\-]?(?:oz|ounces)[^\w]{0,3}(?:capacity))|()'
dfs['m_name']=dfs['product_name'].apply(lambda x: re_extract(pat, str(x)))
dfs['m_long']=dfs['long_desc'].apply(lambda x: re_extract(pat, str(x)))
dfs['m_custom']=dfs['custom_fields'].apply(lambda x: re_extract(pat, str(x)))
oz=dfs[(dfs['m_name'].astype(str)!='[]')|(dfs['m_long'].astype(str)!='[]')|(dfs['m_custom'].astype(str)!='[]')]
print(len(dfs))
print(len(oz))


pat=r'(?i)(\d+(?:\.\d+)?[\s\-]?(?:oz|ounces))|()'
dfs['m_name']=dfs['product_name'].apply(lambda x: re_extract(pat, str(x)))
dfs['m_long']=dfs['long_desc'].apply(lambda x: re_extract(pat, str(x)))
dfs['m_custom']=dfs['custom_fields'].apply(lambda x: re_extract(pat, str(x)))
na=dfs[(dfs['m_name'].astype(str)=='[]')&(dfs['m_long'].astype(str)=='[]')&(dfs['m_custom'].astype(str)=='[]')]


na[curation_col]='n/a'
print(len(na))
matchna_capacity_oz=na[['external_id',curation_col]]

150
14
26


In [40]:
metric='oz'
range_string = """
0 - 1.9 oz
0 - 7.9 oz
0 - 7.9 oz.
16 - 23.9 oz
16 - 23.9 oz.
2 - 3.9 oz
24 - 31.9 oz
24 - 31.9 oz.
32 - 39.9 oz
32 - 39.9 oz.
40 - 47.9 oz
40 - 47.9 oz.
4 - 7.9 oz
48 - 55.9 oz
48 - 55.9 oz.
48 oz and Greater
48 oz. and Greater
56 - 63.9 oz
56 - 63.9 oz.
64 - 71.9 oz
64 - 71.9 oz.
72 - 79.9 oz
72 - 79.9 oz.
80 oz or Greater
80 oz. or Greater
8 - 15.9 oz
8 - 15.9 oz.
 """

range_params = {}
for range_entry in range_string.split('\n'):
    range_nums = re.findall('\d+', range_entry)
    if len(range_nums) > 0: 
        range_params[tuple(map(int, range_nums))] = range_entry.strip()


def  range_app(num_lst):
    updated_labels = []
    for num in num_lst:
        num = float(num)
        for range_param, range_label in range_params.items():
            if len(range_param) == 1:
                if num >= range_param[0]:
                    updated_labels.append(range_label)
            else:
                if num >= range_param[0] and num <= range_param[1]:
                    updated_labels.append(range_label)
    return updated_labels

def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

In [41]:
name=oz[(oz['m_name'].astype(str)!='[]')&(oz['m_long'].astype(str)=='[]')&(oz['m_custom'].astype(str)=='[]')]
long=oz[(oz['m_name'].astype(str)=='[]')&(oz['m_long'].astype(str)!='[]')&(oz['m_custom'].astype(str)=='[]')]
custom=oz[(oz['m_name'].astype(str)=='[]')&(oz['m_long'].astype(str)=='[]')&(oz['m_custom'].astype(str)!='[]')]

custom=oz[(oz['m_custom'].astype(str)!='[]')]
print(len(custom))
custom['Q:capacity_oz']=custom['m_custom'].apply(lambda x: re.sub(r'(?i)(\s?capacity\s?)|()','',str(x))).apply(lambda x: re.sub(r'(?i)(\s?ounces\s?)|(\s?oz\s?\.?)','',str(x))).apply(lambda x: re.sub(r"((?<=\[)\s?'\s?)|(\s?'\s?(?=\]))|((?<=\,)\s?')|('\s?(?=\,))",'"',str(x)))
custom['Q:capacity_oz']=custom['Q:capacity_oz'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(range_app).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x))).apply(lambda x: re.sub(r'\,\s',',',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])) .apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))
rounding_oz(custom, curation_col,'bcfh-mqsu-y')
custom=custom[custom['rounding'].astype(str)=='[]']
print(len(custom))


14
0


In [42]:
print(len(custom))
rounding_oz(custom, curation_col,'a-np-y')
custom=custom[custom['rounding'].astype(str)=='[]']
print(len(custom))
match_capacity_oz=custom[['external_id','Q:capacity_oz']]

0
0


In [43]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matchna_capacity_oz.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribute,matchna_capacity_oz,today)

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_capacity_oz.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribute,match_capacity_oz,today)

[]
[]


# Capacity Gallons

In [44]:
customer_id = '5'
formatted_attribute = 'capacity gal'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [45]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)
# reg_ex(df)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df=df[(df['external_id'].astype(str)!='200075103')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')&(df['external_id'].astype(str)!='200046141')&(df['external_id'].astype(str)!='200046271')]
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

77
[]


In [46]:
pat=r'(?i)(gal.{0,3}\d+)|(capacity)|(\d+.{0,3}gal)'
df['name']=df['product_name'].apply(lambda x: re_extract(pat, str(x)))
df['desc']=df['long_desc'].apply(lambda x: re_extract(pat, str(x)))
dfna=df[(df['name'].astype(str)=='[]')&(df['desc'].astype(str)=='[]')]
dfna[curation_col]='n/a'
matchna_capacity_gal_bbby=dfna[['external_id',curation_col]]

In [47]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribute,matchna_capacity_gal_bbby,today)

[]


In [48]:
customer_id = '5'
formatted_attribute = 'volume_oz'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [49]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)
# reg_ex(df)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df=df[(df['external_id'].astype(str)!='200075103')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')&(df['external_id'].astype(str)!='200046141')&(df['external_id'].astype(str)!='200046271')]
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

75
[]


In [50]:
# NA
pat=r'(?i).{0,10}\d+\.?\d*.{0,3}(?:oz|ounce).{0,10}'
df['long']=df['long_desc'].apply(lambda x: re_extract(pat, str(x)))
print(len(df[df['long'].astype(str)=='[]']))
na=df[df['long'].astype(str)=='[]']
na[curation_col]='n/a'
matchna_volume_bbby=na[['external_id',curation_col]]

40


In [51]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Bed Bath & Beyond', attribute,matchna_volume_bbby,today)

[]


In [52]:
# print(len(df[df['long'].astype(str)!='[]']))
# df[df['long'].astype(str)!='[]']

# Container Store

In [53]:
customer_id = '77'
customer_name='%containerstore%'
formatted_attribute = 'diameter_in'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(dfs))
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['External_id']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)
print(len(df))
diameter_in=df

0


KeyError: "None of [Index(['External_id'], dtype='object')] are in the [columns]"

In [ ]:
# NA
pat=r'(?i)(\d"?.{0,3}x)|(x.{0,3}\d)|(\d.{0,3}dia)'
diameter_in['long']=diameter_in['long_desc'].apply(lambda x: re_extract(pat, str(x)))
print(len(diameter_in[diameter_in['long'].astype(str)=='[]']))
na=diameter_in[diameter_in['long'].astype(str)=='[]']
na[curation_col]='n/a'
matchna_cs_diameter=na[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/Container_Store - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
looks_good('Container Store', attribute,matchna_cs_diameter,today)

In [ ]:
customer_id = '77'
customer_name='%containerstore%'
formatted_attribute = 'height_in'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['Height']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)
print(len(df))
height_in=df

In [ ]:
pat=r'(?i)(\d+(?:.?\d+\/\d+|\.\d+)?"?.?h)|()'
height_in['long']=height_in['product_name'].apply(lambda x: re_extract(pat, str(x))).apply(lambda x: re.sub(r'(?i)("\s?h)','',str(x)))
pat=r'(?i)(\d)'
height_in['heights']=height_in['Height'].apply(lambda x: re_extract(pat, str(x)))
na=height_in[(height_in['long'].astype(str)=='[]')&(height_in['heights'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna_cs_height=na[['external_id',curation_col]]

In [ ]:
height_ins=df[(df['long'].astype(str)!='[]')|(df['heights'].astype(str)!='[]')]
curate(height_ins,'long',attribute)
print(len(height_ins))

In [ ]:
rounding_inch_feet(height_ins, curation_col,'a-eghj-mo-su-z')
print(len(height_ins))
height_ins=height_ins[height_ins['rounding'].astype(str)=='[]']
print(len(height_ins))
match_df_height_cs=height_ins[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/Container_Store - {attribute}-{get_df_name(matches)}-{today}-matches_na.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Container Store', attribute,matchna_cs_height,today)

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/Container_Store - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Container Store', attribute,match_df_height_cs,today)

In [ ]:
Jeff is combining capacity and capacity lbs

In [ ]:
customer_id = '77'
customer_name='%containerstore%'
formatted_attribute = 'capacity_lbs'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['Height']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
df.drop_duplicates(subset='external_id', inplace=True)
print(len(df))
capacity_lbs=df

In [ ]:
# NA
pat=r'(?i)((?:capacity:.?)?\d+(?:\.\d+)?.?lbs?)|()'
capacity_lbs['long']=capacity_lbs['long_desc'].apply(lambda x: re_extract(pat, str(x)))
capacity_lbs[curation_col]=capacity_lbs['long'].apply(lambda x: re.sub(r'(?i)(capacity\:\s?)','',str(x))).apply(lambda x: re.sub(r"(?<=\[)\s?'|(?<=\,)\s?'|'\s?(?=\])|'\s?(?=\,)",'"',str(x)))
capacity_lb=capacity_lbs[capacity_lbs[curation_col].astype(str)!='[]']



na=df[df['long'].astype(str)=='[]']
print(len(na))
na[curation_col]='n/a'
matchna_container_store=na[['external_id',curation_col]]

In [ ]:
rounding_lbs(capacity_lb,curation_col,'ac-km-z') 
print(len(capacity_lb))
capacity_lb=capacity_lb[capacity_lb['rounding'].astype(str)=='[]']
print(len(capacity_lb))
match_df_capacity_lbs_cs=capacity_lb[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/Container_Store - {attribute}-{get_df_name(matches)}-{today}-matches_na.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Container Store', attribute,matchna_container_store,today)

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/Container_Store - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Container Store', attribute,match_df_capacity_lbs_cs,today)

# Dollar Tree-Volume Oz

In [ ]:
customer_id = '135'
customer_name='%bedbathandbeyond%'
dateszs='2002-08-11'
attribut='volume_oz'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribute':attribut}

dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(dfs))
# dfz=dfs[dfs['custom_fields'].astype(str)!=None]
# custom_field_df=pd.json_normalize(dfs['custom_fields'])
# fields = ['LONG_DESCRIPTION']
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# df.drop_duplicates(subset='external_id', inplace=True)

In [ ]:
pat='''((?:\d+\.)?\d.{0,2}(?:oz|ounc))'''
oz,na=three(dfs,pat)

In [ ]:
name=oz[oz['m_name'].astype(str)!='[]']
print(len(name))
other=oz[(oz['m_name'].astype(str)=='[]')&((oz['m_desc'].astype(str)!='[]')|(oz['m_custom'].astype(str)!='[]'))]
print(len(other))

In [ ]:
na[curation_col]='n/a'
match_na_oz=na[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_na_oz.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(match_na_oz['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_na_oz,today)

In [ ]:
name['buckets'].explode().value_counts()

In [ ]:
def bucketz(name,bucket,range_string,regex):
    # bucket gum
    Chocolate=name[name['buckets'].astype(str)==bucket]
    print(len(Chocolate))

    metric='oz'


    range_params = {}
    for range_entry in range_string.split('\n'):
        range_nums = re.findall('\d+', range_entry)
        if len(range_nums) > 0: 
            range_params[tuple(map(int, range_nums))] = range_entry.strip()


    def  range_app(num_lst):
#         print(num_lst)
        updated_labels = []
        for num in num_lst:
            num = float(num)
            for range_param, range_label in range_params.items():
#                 print(range_param)
                if len(range_param) == 1:
                    if num >= range_param[0]:
                        updated_labels.append(range_label)
                else:
                    if num >= range_param[0] and num <= range_param[1]:
                        updated_labels.append(range_label)
#         print(updated_labels)
        return updated_labels

    def remove_duplicates(A):
        [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
        return A

    Chocolate[curation_col]=Chocolate['m_name'].apply(lambda x: re.sub(fr'(?i){regex}','',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(range_app).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x))).apply(lambda x: re.sub(r'\,\s',',',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: '["'+'","'.join(natsorted(x))+'"]').apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))
    return Chocolate

In [ ]:
range_string = """
    0 - 2.9 oz
    3 - 5.9 oz
    6 - 8.9 oz
    9 - 11.9 oz
    12 - 14.9 oz
    15 - 23.9 oz
    24 - 31.9 oz
    32 - 39.9 oz
    40 - 47.9 oz
    48 - 55.9 oz
    56 - 63.9 oz
    64 - 71.9 oz
    72 - 79.9 oz
    80 oz or Greater
    n/a"""

In [ ]:
name[curation_col]=name['m_name'].apply(lambda x: re.sub(r"((?:\-oz|(?<=\d)oz))",' oz',str(x))).apply(lambda x: re.sub(r"(\.0(?=\s))|(\.00(?=\s))",'',str(x))).apply(lambda x: re.sub(r"\s?'\s?",'"',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x))).apply(lambda x: re.sub(r'\,\s',',',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: '["'+'","'.join(natsorted(x))+'"]').apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))
name[curation_col].explode().value_counts()

In [ ]:
print(len(name))
rounding_oz(name, curation_col,'a-np-y')
name=name[name['rounding'].astype(str)=='[]']
print(len(name))
Gum=bucketz(name,'Gum, Candies, & Mints',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
match_Gum=Gum[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Gum.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Gum,today)

In [ ]:
# Chocolate=bucketz(name,'Chocolate',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_Chocolate=Chocolate[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Chocolate.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Chocolate,today)

In [ ]:
# artistic=bucketz(name,'Artistic Crafting Supplies',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_artistic=artistic[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_artistic.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_artistic,today)

In [ ]:
# Cleaners=bucketz(name,'Cleaners & Polishes',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_Cleaners=Cleaners[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Cleaners.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Cleaners,today)

In [ ]:
# Nut=bucketz(name,'Nuts, Seeds, & Legumes',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_Nut=Nut[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Nut.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Nut,today)

In [ ]:
# Prepared=bucketz(name,'Prepared Soft Drinks & Water',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_Prepared=Prepared[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Prepared.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Prepared,today)

In [ ]:
# Sauces=bucketz(name,'Sauces, Pastes, & Gravies',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_Sauces=Sauces[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Sauces.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Sauces,today)

In [ ]:
# Fruit=bucketz(name,'Fruit Snacks',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_Fruit=Fruit[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Fruit.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Fruit,today)

In [ ]:
# Shredded=bucketz(name,'Shredded & Tissue Papers',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_Shredded=Shredded[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Shredded.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Shredded,today)

In [ ]:
# Deodorant=bucketz(name,'Deodorants & Antiperspirants',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_Deodorant=Deodorant[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Deodorant.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Deodorant,today)

In [ ]:
range_string = """
    0 - 0.25 oz
    0.26 - 0.5 oz
    0.51 - 0.75 oz
    0.76 - 1.0 oz
    1.1 - 1.5 oz
    1.6 - 2.0 oz
    2.1 - 2.5 oz
    2.6 - 3.0 oz
    3.1 - 5.9 oz
    6.0 - 9.9 oz
    10.0 oz and Greater
    n/a
     """

In [ ]:
# herb=bucketz(name,'Herbs, Spices, & Seasonings',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_herb=herb[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_herb.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_herb,today)

In [ ]:
range_string = """
    0 - 2.9 oz
    3 - 5.9 oz
    6 - 8.9 oz
    9 - 11.9 oz
    12 - 14.9 oz
    15 - 23.9 oz
    24 - 31.9 oz
    32 - 39.9 oz
    40 - 47.9 oz
    48 oz and Greater
    n/a
     """

In [ ]:
# meat=bucketz(name,'Meats & Meat Alternatives',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_meat=meat[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_meat.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_meat,today)

In [ ]:
# Soups=bucketz(name,'Soups & Broths',range_string,'''((?:\-|\s)?(?:oz|ounc))''')
# match_Soups=Soups[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/Dollar Tree - {attribute}-{get_df_name(matches)}-{today}-match_Soups.csv',index=False) 
    import collections
# print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# looks_good('Dollar Tree', attribut,match_Soups,today)